In [19]:
import torch
import csv
import json
import os
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize 
from tqdm import tqdm
import math


In [20]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mrmrk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
class RemoveOnIterate:
    def __init__(self, data):
        self.data = data

    def __iter__(self):
        # This method should return the iterator object (in this case, self)
        return self

    def __next__(self):
        # Check if there are more elements in the data
        if self.data:
            # Get and remove the next element from the data
            result = self.data.pop(0)
            return result
        else:
            # No more elements, raise StopIteration
            raise StopIteration

In [22]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [23]:
base_path = r"C:\Users\mrmrk\OneDrive\Documents\GitHub\recipe_generation\\"
original_cvs_name = r"RecipeNLG_dataset.csv"
prepeard_data_name = "prepeard_data.csv"
ingredient_vocab_name = "ingredient_vocab.csv"
recipie_vocab_name = "recipie_vocab.csv"

In [24]:
bads = ['\x00']
data = []

def fix_text(line, j):
    return j.join([r for r in json.loads(line) if not all([r.__contains__(bad) for bad in bads])])

with open(base_path + original_cvs_name, 'r', newline='', encoding='utf-8') as input_csvfile:
        csv_reader = csv.reader(input_csvfile)
        next(csv_reader, None)

        # Copy selected columns from the input file to the output file
        for i,row in enumerate(tqdm(csv_reader)):
            data.append([fix_text(row[2], " * "),fix_text(row[3], " ")])
            if i == 10000:
                pass
data[0]

229650it [00:04, 56312.21it/s]

In [ ]:
print(word_tokenize(data[0][0]))
print(word_tokenize(data[0][1]))

['1', 'c.', 'firmly', 'packed', 'brown', 'sugar', '*', '1/2', 'c.', 'evaporated', 'milk', '*', '1/2', 'tsp', '.', 'vanilla', '*', '1/2', 'c.', 'broken', 'nuts', '(', 'pecans', ')', '*', '2', 'Tbsp', '.', 'butter', 'or', 'margarine', '*', '3', '1/2', 'c.', 'bite', 'size', 'shredded', 'rice', 'biscuits']
['In', 'a', 'heavy', '2-quart', 'saucepan', ',', 'mix', 'brown', 'sugar', ',', 'nuts', ',', 'evaporated', 'milk', 'and', 'butter', 'or', 'margarine', '.', 'Stir', 'over', 'medium', 'heat', 'until', 'mixture', 'bubbles', 'all', 'over', 'top', '.', 'Boil', 'and', 'stir', '5', 'minutes', 'more', '.', 'Take', 'off', 'heat', '.', 'Stir', 'in', 'vanilla', 'and', 'cereal', ';', 'mix', 'well', '.', 'Using', '2', 'teaspoons', ',', 'drop', 'and', 'shape', 'into', '30', 'clusters', 'on', 'wax', 'paper', '.', 'Let', 'stand', 'until', 'firm', ',', 'about', '30', 'minutes', '.']


In [ ]:
ingredients_count = Counter()
recipies_count = Counter()
data_tokanized = []
for (ingrent_list, recipie) in tqdm(RemoveOnIterate(data)):
    ingrent_tokalized = [w.lower() for w in word_tokenize(ingrent_list)]
    for word in ingrent_tokalized:
        ingredients_count[word] += 1
    recipie_tokalized = [w.lower() for w in word_tokenize(recipie)]
    for word in recipie_tokalized:
        recipies_count[word] += 1
    data_tokanized.append((ingrent_tokalized, recipie_tokalized))

 40%|███▉      | 885836/2231142 [12:02<18:17, 1225.53it/s] 


KeyboardInterrupt: 

In [ ]:
print(recipies_count.most_common(20))

In [ ]:
print(data_tokanized)

In [ ]:
1/20

In [ ]:
ing_sum = sum(ingredients_count.values())
rec_sum = sum(recipies_count.values())
print(ing_sum, rec_sum)
ing_prob = 0.01/100
rec_prob = 0.01/100
ingredient_words = ["<UKN>"] + [w for w, c in ingredients_count.most_common() if c/ing_sum >= ing_prob]
recipie_words = ["<UKN>"] + [w for w, c in recipies_count.most_common() if  c/rec_sum >= rec_prob]
print(len(ingredient_words), ingredient_words)

In [ ]:
def get_id(vocab, word):
    try:
        return vocab.index(word)
    except:
        return 0
data_ids = []
for ing, rec in tqdm(RemoveOnIterate(data_tokanized)):
    ing_ids = [get_id(ingredient_words, i) for i in ing]
    rec_ids = [get_id(recipie_words, r) for r in rec]
    data_ids.append((ing_ids, rec_ids))

In [ ]:


if not os.path.exists(base_path + prepeard_data_name):
    with open(base_path + prepeard_data_name, 'w', newline='', encoding='utf-8') as output_csvfile:
        csv_writer = csv.writer(output_csvfile)

        csv_writer.writerow(["X","Y"])

        # Copy selected columns from the input file to the output file
        for row in data_ids:
            csv_writer.writerow(row)

In [ ]:


if not os.path.exists(base_path + ingredient_vocab_name):
    with open(base_path + ingredient_vocab_name, 'w', newline='', encoding='utf-8') as output_csvfile:
        csv_writer = csv.writer(output_csvfile)

        csv_writer.writerow(["words"])

        # Copy selected columns from the input file to the output file
        for row in ingredient_words:
            csv_writer.writerow([row])

In [ ]:
if not os.path.exists(base_path + recipie_vocab_name):
    with open(base_path + recipie_vocab_name, 'w', newline='', encoding='utf-8') as output_csvfile:
        csv_writer = csv.writer(output_csvfile)

        csv_writer.writerow(["wors"])

        # Copy selected columns from the input file to the output file
        for row in recipie_words:
            csv_writer.writerow([row])